In [69]:
import os, sys, json

import numpy as np

import pandas as pd


from IPython import InteractiveShell

InteractiveShell.ast_node_interactivity="all"

import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  0.1.59


In [70]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, ws.location, sep = '\t')

Found the config file in: C:\Users\k.shridhar\Desktop\Avanade\Analytics\openhack-sg-2018\config.json
openhack-challenge5	southeastasia	openhack-sg-kishan	southeastasia


In [71]:
experiment_name = 'challenge5v1'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [72]:
exp

Name,Workspace,Report Page,Docs Page
challenge5v1,openhack-challenge5,Link to Azure Portal,Link to Documentation


#### REGISTER MODEL IN WORKSPACE

In [159]:
from azureml.core.model import Model
import sklearn

library_version = "sklearn"+sklearn.__version__.replace(".","x")

model_path=os.path.join(os.getcwd(),"outputs","challenge5_model.pkl")

model_path
model = Model.register(model_path = model_path,
                       model_name = "challenge5",
                       tags = {'area': "challenge 5", 'type': "classification", 'version': library_version},
                       description = "Classification model to predict image categories (12 of them)",
                       workspace = ws)


'C:\\Users\\k.shridhar\\Desktop\\Avanade\\Analytics\\openhack-sg-2018\\outputs\\challenge5_model.pkl'

Registering model challenge5


#### CREATE DOCKER IMAGE OF MODEL

In [157]:
# import requests
# from PIL import Image
# from io import BytesIO
# import matplotlib.image as img

# def resize_image(image_url,size=128):
#     '''Resize an Image from URL'''
#     response = requests.get('https://i.pinimg.com/originals/0f/2d/8d/0f2d8d53c8be1353f92f64d76cdd6657.jpg')
#     im=Image.open(BytesIO(response.content))
#     old_size = im.size
#     ratio = float(size)/max(old_size)
#     new_size = tuple([int(x*ratio) for x in old_size])
#     imres=im.copy()
#     imres.thumbnail(new_size,Image.ANTIALIAS)     
#     new_im = Image.new("RGB", (size, size),color='white')
#     new_im.paste(imres)
    
#     return new_im

# new_im=resize_image('https://i.pinimg.com/originals/0f/2d/8d/0f2d8d53c8be1353f92f64d76cdd6657.jpg')

# img.imread(new_im)

TypeError: Object does not appear to be a 8-bit string path or a Python file-like object

In [180]:
%%writefile score.py
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('challenge5')
    model = joblib.load(model_path)

def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


In [161]:
# response = requests.get('https://i.pinimg.com/originals/0f/2d/8d/0f2d8d53c8be1353f92f64d76cdd6657.jpg')
# from io import BytesIO
# im=Image.open(BytesIO(response.content))
# old_size = im.size
# ratio = float(size)/max(old_size)
# new_size = tuple([int(x*ratio) for x in old_size])
# imres=im.copy()
# imres.thumbnail(new_size,Image.ANTIALIAS)     



In [181]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','scikit-image'])

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

524

In [182]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
  - azureml-defaults==0.1.59
- numpy
- scikit-learn
- scikit-image



In [183]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(runtime= "python",
                                 execution_script="score.py",
                                 conda_file="myenv.yml",
                                 tags = {'type': "classification"},
                                 description = "Image classification model for challenge 5")

image = Image.create(name = "challenge5image",
                     # this is the model object 
                     models = [model],
                     image_config = image_config, 
                     workspace = ws)

Creating image


In [184]:
image.wait_for_creation(show_output = True)


Running.................................................
SucceededImage creation operation finished for image challenge5image:1, operation "Succeeded"


#### CREATE CONFIGURATION FILE FOR ACI WEB SERVICE

In [185]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Image Classification",  "method" : "Sklearn Model"}, 
                                               description='Perform Image classification with Scikit-learn')


#### CREATE WEB SERVICE

In [186]:
from azureml.core.webservice import Webservice

aci_service_name = 'image-classify'
print(aci_service_name)
aci_service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                           image = image,
                                           name = aci_service_name,
                                           workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)

image-classify
Creating service
Running....................................................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


In [187]:
print(aci_service.scoring_uri)

http://13.76.193.235:80/score


In [304]:
X_new=[]

def prepare_modeldata():
    '''Function to read in a directory of files and label them as Y while the pixels are X'''

    path=os.path.join(os.getcwd(),'challenge3')
    for files in os.listdir(path):
          print(files)
          import matplotlib.image as img
          image = img.imread(os.path.join(path,files))
          X_new.append(image)

    return X_new


X_new=prepare_modeldata()


899119.jpeg
T1.jpg
T2.jpg
T3.jpg
T4.jpg
T5.jpg


In [305]:

X_new=(np.array(X_new))

X_new.shape
X_new=X_new/255


(6, 128, 128, 3)

#### TEST WEB SERVICE

In [318]:
url=aci_service.scoring_uri

headers={'content-type':'application/json'}

with open(img_path,'r') as img_file:
    X_test=np.array(img_file)

for t in X_new:
    res=requests.post(url=url,headers=headers,data=json.dumps({"data":t.reshape(1,-1).tolist()}))

    res.json()



'[7]'

'[4]'

'[0]'

'[6]'

'[11]'

'[5]'